In [23]:
import pandas as pd
from tensorflow.keras.layers import LSTM,Dense,Dropout,Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import re
import numpy as np

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
url = 'Fake.csv'
data = pd.read_csv(url)

In [7]:
data.shape

(20800, 5)

In [8]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
data.dropna(inplace=True)

In [10]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
data.shape

(18285, 5)

In [12]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
sw = stopwords.words('english')
lem = WordNetLemmatizer()
def clean_data(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]',' ',text)
    text = re.sub(r'\b\w{1,2}\b','',text)
    text = re.sub(r'[\s+]',' ',text)
    text = ' '.join(lem.lemmatize(word) for word in text.split() if not word in sw)
    return text

In [14]:
l=[]
for i in data.title.apply(lambda x : clean_data(x)):
  l.append(i)

In [16]:
voc_size = 5000
total = [one_hot(word,voc_size) for word in l]
len(total[0])

10

In [41]:
sequence = pad_sequences(total, padding = 'pre', maxlen = 20)
sequence

array([[   0,    0,    0, ..., 3235,   43, 2149],
       [   0,    0,    0, ..., 2011, 3983,   25],
       [   0,    0,    0, ..., 4189, 2576, 4990],
       ...,
       [   0,    0,    0, ...,  153, 3238,   31],
       [   0,    0,    0, ...,  747, 2577, 4751],
       [   0,    0,    0, ...,    0, 1483, 1383]], dtype=int32)

In [42]:
y = np.array(data.label)

In [50]:
model = Sequential()

model.add(Embedding(voc_size,40,input_length=20))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer = 'adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [51]:
X_train,X_test,y_train,y_test = train_test_split(sequence,y,test_size=0.3,random_state=42)

In [53]:
result = model.fit(X_train,y_train,
                   epochs=100,
                   batch_size=64,
                   verbose=2,
                   validation_data = (X_test,y_test))

Epoch 1/100
200/200 - 34s - loss: 0.3496 - accuracy: 0.8370 - val_loss: 0.2092 - val_accuracy: 0.9134
Epoch 2/100
200/200 - 1s - loss: 0.1465 - accuracy: 0.9412 - val_loss: 0.1973 - val_accuracy: 0.9163
Epoch 3/100
200/200 - 1s - loss: 0.1012 - accuracy: 0.9633 - val_loss: 0.2591 - val_accuracy: 0.9182
Epoch 4/100
200/200 - 1s - loss: 0.0786 - accuracy: 0.9729 - val_loss: 0.2500 - val_accuracy: 0.9127
Epoch 5/100
200/200 - 1s - loss: 0.0605 - accuracy: 0.9801 - val_loss: 0.3198 - val_accuracy: 0.9079
Epoch 6/100
200/200 - 1s - loss: 0.0502 - accuracy: 0.9841 - val_loss: 0.3399 - val_accuracy: 0.9076
Epoch 7/100
200/200 - 1s - loss: 0.0336 - accuracy: 0.9898 - val_loss: 0.3865 - val_accuracy: 0.9019
Epoch 8/100
200/200 - 1s - loss: 0.0250 - accuracy: 0.9926 - val_loss: 0.3776 - val_accuracy: 0.9081
Epoch 9/100
200/200 - 1s - loss: 0.0242 - accuracy: 0.9927 - val_loss: 0.4202 - val_accuracy: 0.9072
Epoch 10/100
200/200 - 1s - loss: 0.0167 - accuracy: 0.9950 - val_loss: 0.5765 - val_accur